In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV,train_test_split, KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.datasets import fetch_openml


In [2]:
# Cargar el conjunto de datos MNIST
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist["data"], mnist["target"]

# Normalizar las imágenes de 0 a 1
X = X.astype('float32') / 255.0

X=X.values
# Convertir las etiquetas a enteros y luego a categóricas (one-hot encoding)
y = to_categorical(y.astype('int'))



h:\Anaconda\envs\IA\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
type(X)

numpy.ndarray

In [4]:
type(y)

numpy.ndarray

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
def create_model(optimizer='adam', init='glorot_uniform', activation='relu'):
    model = Sequential()
    model.add(Flatten(input_shape=(784,)))
    model.add(Dense(128, kernel_initializer=init, activation=activation))
   # model.add(Dense(64, kernel_initializer=init, activation=activation))
    model.add(Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model



In [7]:
def manual_grid_search(X_train_k, y_train_k, X_val_k, y_val_k, param_grid):
    for batch in param_grid['batch_size']:
        model=create_model()
        history=model.fit(X_train_k, y_train_k,validation_data=(X_val_k, y_val_k),batch_size=batch, epochs=2,verbose=1)

    return batch,history.history['val_accuracy'][-1]


In [8]:
# Define the grid of hyperparameters
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [10],
    'optimizer': ['adam'],
    'init': ['normal'],
    'activation': ['relu', 'tanh']
}

# Implement K-Fold Cross-Validation
kf = KFold(n_splits=3)

best_params = None
best_score = 0

for train_index, val_index in kf.split(X_train):
    X_train_k, X_val_k = X_train[train_index], X_train[val_index]
    y_train_k, y_val_k = y_train[train_index], y_train[val_index]
    
    params, score = manual_grid_search(X_train_k, y_train_k, X_val_k, y_val_k, param_grid)
    print(score)
    if score > best_score:
        best_score = score
        best_params = params

print(f"\nBest Score: {best_score:.4f} with Best Params: {best_params}")

Epoch 1/2
1167/1167 [==============================] - 6s 4ms/step - loss: 0.3061 - accuracy: 0.9142 - val_loss: 0.1740 - val_accuracy: 0.9505
Epoch 2/2
1167/1167 [==============================] - 5s 4ms/step - loss: 0.1387 - accuracy: 0.9594 - val_loss: 0.1343 - val_accuracy: 0.9594
Epoch 1/2
584/584 [==============================] - 3s 4ms/step - loss: 0.3640 - accuracy: 0.8967 - val_loss: 0.2191 - val_accuracy: 0.9366
Epoch 2/2
584/584 [==============================] - 2s 4ms/step - loss: 0.1694 - accuracy: 0.9508 - val_loss: 0.1674 - val_accuracy: 0.9502
Epoch 1/2
292/292 [==============================] - 2s 4ms/step - loss: 0.4455 - accuracy: 0.8783 - val_loss: 0.2451 - val_accuracy: 0.9286
Epoch 2/2
292/292 [==============================] - 1s 3ms/step - loss: 0.2022 - accuracy: 0.9426 - val_loss: 0.1854 - val_accuracy: 0.9443
0.9442867040634155
Epoch 1/2
1167/1167 [==============================] - 4s 4ms/step - loss: 0.3105 - accuracy: 0.9118 - val_loss: 0.1708 - val_accur

In [11]:
print(f"\nBest Score: {best_score:.4f} with Best Params: {best_params}")


Best Score: 0.9479 with Best Params: 128


In [ ]:
best_params

In [10]:
# Train the final model using the best hyperparameters
final_model = create_model(optimizer=best_params['optimizer'], 
                           init=best_params['init'], 
                           activation=best_params['activation'])

final_model.fit(X_train, y_train, 
                epochs=best_params['epochs'], 
                batch_size=best_params['batch_size'], 
                verbose=1)

# Evaluate the final model on the test set
test_score = final_model.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Accuracy: {test_score[1]:.4f}")


TypeError: 'int' object is not subscriptable